<a href="https://colab.research.google.com/github/animalran/csvfiles/blob/master/DCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import pandas as pd
import bs4 as bs
import urllib.request
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
from googlesearch import search

In [0]:
symbol = "BHARTIARTL"
beta=.80

In [0]:
url1a='https://www.screener.in/company/'
url1b='/consolidated'
url1 =url1a+symbol+url1b
url2a='https://www.ndtv.com/business/stock/'
url2b='-ltd_'
url2c='/holdings'
url2 = url2a+symbol+url2b+symbol+url2c

In [90]:
# capex
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:32.0) Gecko/20100101 Firefox/34.0',}
req=requests.get(url1, headers=header)
soup=BeautifulSoup(req.content, 'lxml')
data = soup.find('section', attrs={'id':'cash-flow'})
rows = data.find_all('tr')
rowyear = data.find_all('thead')
res = []
for tr in rowyear:
    th = tr.find_all('th')
    yrow = [tr.text.strip() for tr in th if tr.text.strip()]
res.append(['Index']+yrow)
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res)
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
soup2=BeautifulSoup(req.content, 'lxml')
data2=soup2.find(attrs={"id": "company-info"})
datacomid=data2.get('data-company-id')#flex-row flex-baseline flex-space-between
req2=requests.get('https://www.screener.in/api/company/'+datacomid+'/schedules/?parent=Cash+from+Investing+Activity&section=cash-flow&consolidated=', headers=header)
import json
j = req2.json()
import numpy as np
df2 = pd.DataFrame.from_dict(j)
df2
#df2["Fixed assets purchased"]
capex=df2["Fixed assets purchased"]+df2["Fixed assets sold"]
capex16=int(capex["Mar 2016"].replace(',', ''))/1000
capex17=int(capex["Mar 2017"].replace(',', ''))/1000
capex18=int(capex["Mar 2018"].replace(',', ''))/1000
capex19=int(capex["Mar 2019"].replace(',', ''))/1000
capex19

-29478.428

In [91]:
# cash 
month= 'Mar 2019'
req4=requests.get('https://www.screener.in/api/company/'+datacomid+'/schedules/?parent=Other+Assets&section=balance-sheet', headers=header)
cashjson=req4.json()
cashdf= pd.DataFrame.from_dict(cashjson)
c=cashdf['Cash Equivalents']
cash=c[month]
cash=int(cash.replace(',', ''))
print(cash)
type(cash)

220


str

In [80]:
r1 = requests.get(url1).content
r2 = requests.get(url2).content
table2 = pd.read_html(r2)
table1 = pd.read_html(r1)
tab1 = table1[1]
tab2 = table1[6]
tab3 = table1[7]
tab4 = table2[3]
s = int(tab4.loc[21:21,"Total Number of Shares"])
type(s)

int

In [0]:
month1='Mar 2019'
Int = int(tab1.loc[5:5,month1])
brw = int(tab2.loc[2:2,month1])
#brw=1
ot_liab = int(tab2.loc[3:3,month1])
sh_cap = int(tab2.loc[0:0,month1])
res = int(tab2.loc[1:1,month1])
Tc = .3

In [82]:
#Rates
market_RoR = .15
risk_free = .064
Re = risk_free+beta*(market_RoR-risk_free)
Rd = Int/brw
E = sh_cap + res
#D = brw + ot_liab
D = brw
A = E+D
dis = E/A*Re+(1-Tc)*D/A*Rd
f5 = .14
s5 = .09
tg = .04
print(Re)
print(Rd)
print(dis)

0.1328
0.08780336129093982
0.08735832529227144


In [0]:
# free cash flow calculation = cash flow from operation-CAPEX+interest-tax shield
co16 = int(tab3.loc[0:0,"Mar 2016"])
co17 = int(tab3.loc[0:0,"Mar 2017"])
co18 = int(tab3.loc[0:0,"Mar 2018"])
co19 = int(tab3.loc[0:0,"Mar 2019"])
Int16 = int(tab1.loc[5:5,"Mar 2016"])
Int17 = int(tab1.loc[5:5,"Mar 2017"])
Int18 = int(tab1.loc[5:5,"Mar 2018"])
Int19 = int(tab1.loc[5:5,"Mar 2019"])
shield16 = Int16*Tc
shield17 = Int17*Tc
shield18 = Int18*Tc
shield19 = Int19*Tc

In [0]:
fcf16 = co16+capex16+Int16-shield16
fcf17 = co17+capex17+Int17-shield17
fcf18 = co18+capex18+Int18-shield18
fcf19 = co19+capex19+Int19-shield19
FCF = (fcf16+fcf17+fcf18+fcf19)/4

In [0]:
# cash flow for first 5 yr
one = FCF+(FCF*f5)
two = one+(one*f5)
three = two+(two*f5)
four = three+(three*f5)
five = four+(four*f5)
# cash flow for next 5 yr
six = five+(five*s5)
seven = six+(six*s5)
eight = seven+(seven*s5)
nine = eight+(eight*s5)
ten = nine+(nine*s5)
# terminal value
t1 = ten+(ten*tg)
t2 = (dis-tg)
t = t1/t2

In [86]:
#present value of 10yrs
p1 = one/(1+dis)**1
p2 = two/(1+dis)**2
p3 = three/(1+dis)**3
p4 = four/(1+dis)**4
p5 = five/(1+dis)**5
p6 = six/(1+dis)**6
p7 = seven/(1+dis)**7
p8 = eight/(1+dis)**8
p9 = nine/(1+dis)**9
p10 = ten/(1+dis)**10
pt = t/(1+dis)**10
PV = p1+p2+p3+p4+p5+p6+p7+p8+p9+p10+pt
type(cash)

int

In [0]:
# Net DEBT
net_debt=brw-cash

In [88]:
numerator=PV-net_debt
share_price=(numerator*10000000)/s
print(share_price)


-14374.372686117573
